# AI in Healthcare - Assignment 2: MIMIC-III SQL

### Connect to MIMIC-III Through Google BigQuery

In [ ]:
# Auth is meant for Google Colab. May have to set up dataset locally or create your own Big Query dataset
# Standard Jupyter Notebook may have a different setup process
from google.colab import auth
from google.cloud import bigquery
bq_client = bigquery.Client(project = "your_project_id")
auth.authenticate_user()


In [ ]:
## Import pandas
import pandas as pd


### Query 1 - Parkinson's by Gender

In [ ]:
bq_client.query('''
SELECT gender, COUNT(*) AS num_patients
FROM `physionet-data.mimiciii_clinical.patients`
WHERE subject_id IN (
    SELECT subject_id FROM physionet-data.mimiciii_clinical.diagnoses_icd WHERE icd9_code = '3320'
)
GROUP BY gender;
''').to_dataframe()


,gender,num_patients
0,F,143
1,M,254


### Query 2 - Parkinson's by Ethnicity

In [ ]:
bq_client.query('''
WITH ethnicity_distribution AS (
    SELECT
        ethnicity,
        COUNT(DISTINCT subject_id) AS num_patients
    FROM physionet-data.mimiciii_clinical.admissions
    WHERE subject_id IN (
        SELECT DISTINCT subject_id
        FROM physionet-data.mimiciii_clinical.diagnoses_icd
        WHERE icd9_code = '3320'
    )
    GROUP BY ethnicity
),
total_ethnicity_admissions AS (
    SELECT
        ethnicity,
        COUNT(DISTINCT subject_id) AS total_admissions
    FROM physionet-data.mimiciii_clinical.admissions
    GROUP BY ethnicity
)
SELECT
    e.ethnicity,
    e.num_patients,
    t.total_admissions,
    (e.num_patients / t.total_admissions) AS percentage_for_ethnicity
FROM ethnicity_distribution e
JOIN total_ethnicity_admissions t ON e.ethnicity = t.ethnicity
ORDER BY percentage_for_ethnicity DESC;
''').to_dataframe()


,ethnicity,num_patients,total_admissions,percentage_for_ethnicity
0,ASIAN - JAPANESE,1,7,0.142857
1,HISPANIC/LATINO - SALVADORAN,1,15,0.066667
2,ASIAN - CHINESE,6,231,0.025974
3,AMERICAN INDIAN/ALASKA NATIVE,1,45,0.022222
4,MULTI RACE ETHNICITY,2,112,0.017857
5,ASIAN - ASIAN INDIAN,1,58,0.017241
6,WHITE - OTHER EUROPEAN,1,71,0.014085
7,BLACK/HAITIAN,1,75,0.013333
8,HISPANIC/LATINO - PUERTO RICAN,2,156,0.012821
9,WHITE,317,32192,0.009847


### Query 3 - Patient's with Parkinson's Age Distribution at First Admission

In [ ]:
bq_client.query('''
WITH first_admission AS (
    SELECT
        p.subject_id,
        DATE_DIFF(DATE(MIN(a.admittime)),
                  ARRAY_AGG(p.dob ORDER BY a.admittime LIMIT 1)[OFFSET(0)],
                  YEAR) AS age_at_first_admission
    FROM physionet-data.mimiciii_clinical.patients p
    JOIN physionet-data.mimiciii_clinical.admissions a
        ON p.subject_id = a.subject_id
    WHERE p.subject_id IN (
        SELECT subject_id FROM physionet-data.mimiciii_clinical.diagnoses_icd
        WHERE icd9_code = '3320'
    )
    GROUP BY p.subject_id -- Grouping by patient to ensure each one is counted once
    HAVING age_at_first_admission <= 110
)
SELECT
    MIN(age_at_first_admission) AS min_age,
    APPROX_QUANTILES(age_at_first_admission, 4)[OFFSET(1)] AS Q1,
    AVG(age_at_first_admission) AS avg_age,
    APPROX_QUANTILES(age_at_first_admission, 4)[OFFSET(3)] AS Q3,
    MAX(age_at_first_admission) AS max_age
FROM first_admission;
''').to_dataframe()


,min_age,Q1,avg_age,Q3,max_age
0,34,70,75.582857,83,89


### Query 4 - Readmissions for Parkinson's Patients

In [ ]:
bq_client.query('''
WITH admission_data AS (
    SELECT subject_id, COUNT(DISTINCT hadm_id) AS total_admissions
    FROM physionet-data.mimiciii_clinical.admissions
    WHERE subject_id IN (
        SELECT subject_id FROM physionet-data.mimiciii_clinical.diagnoses_icd
        WHERE icd9_code = '3320'
    )
    GROUP BY subject_id
)
SELECT
    COUNT(DISTINCT subject_id) AS total_patients,
    SUM(total_admissions) AS total_admissions,
    COUNT(CASE WHEN total_admissions > 1 THEN subject_id END) AS readmitted_patients,
    COUNT(CASE WHEN total_admissions >= 3 THEN subject_id END) AS patients_admitted_atleast_3_times
FROM admission_data;
''').to_dataframe()


,total_patients,total_admissions,readmitted_patients,patients_admitted_atleast_3_times
0,397,595,109,43


### Query 5 - Parkinson's by Readmission Time

In [ ]:
bq_client.query('''
WITH readmission_times AS (
    SELECT
        subject_id,
        TIMESTAMP_DIFF(
            LEAD(admittime) OVER(PARTITION BY subject_id ORDER BY admittime),
            dischtime,
            DAY
        ) AS days_before_readmission
    FROM physionet-data.mimiciii_clinical.admissions
    WHERE subject_id IN (
        SELECT subject_id FROM physionet-data.mimiciii_clinical.diagnoses_icd
        WHERE icd9_code = '3320'
    )
)
SELECT
    AVG(days_before_readmission) AS avg_days_before_readmission
FROM readmission_times
WHERE days_before_readmission IS NOT NULL;
''').to_dataframe()


,avg_days_before_readmission
0,441.075758


### Query 6 - Parkinson's Patients vs. Other Patients Lenth of Stay

In [ ]:
bq_client.query('''
SELECT
    CASE
        WHEN subject_id IN (
            SELECT subject_id FROM physionet-data.mimiciii_clinical.diagnoses_icd
            WHERE icd9_code = '3320'
        ) THEN 'Parkinsons Patients'
        ELSE 'Other Patients'
    END AS patient_group,
    AVG(TIMESTAMP_DIFF(dischtime, admittime, Day)) AS avg_length_of_stay_days
FROM physionet-data.mimiciii_clinical.admissions
GROUP BY patient_group;
''').to_dataframe()


,patient_group,avg_length_of_stay_days
0,Other Patients,10.095802
1,Parkinsons Patients,9.596639


### Query 7 - Top Co-occuring diagnoses with Parkinson's Patients

In [ ]:
bq_client.query('''
SELECT
    d2.icd9_code,
    dicd.long_title,
    COUNT(DISTINCT d2.subject_id) AS co_occurrence_count
FROM physionet-data.mimiciii_clinical.diagnoses_icd d
JOIN physionet-data.mimiciii_clinical.diagnoses_icd d2
    ON d.subject_id = d2.subject_id
JOIN physionet-data.mimiciii_clinical.d_icd_diagnoses dicd
    ON d2.icd9_code = dicd.icd9_code
WHERE d.icd9_code = '3320' AND d2.icd9_code != '3320'
GROUP BY d2.icd9_code, dicd.long_title
ORDER BY co_occurrence_count DESC
LIMIT 10;
''').to_dataframe()


,icd9_code,long_title,co_occurrence_count
0,4019,Unspecified essential hypertension,202
1,4280,"Congestive heart failure, unspecified",132
2,5849,"Acute kidney failure, unspecified",130
3,42731,Atrial fibrillation,128
4,5990,"Urinary tract infection, site not specified",115
5,41401,Coronary atherosclerosis of native coronary ar...,104
6,5070,Pneumonitis due to inhalation of food or vomitus,103
7,51881,Acute respiratory failure,98
8,25000,Diabetes mellitus without mention of complicat...,86
9,2724,Other and unspecified hyperlipidemia,82


### Query 8 - Top Medications Administered to Parkinson's Patients

In [ ]:
bq_client.query('''
SELECT
    pr.drug,
    COUNT(DISTINCT pr.subject_id) AS num_patients_prescribed
FROM physionet-data.mimiciii_clinical.prescriptions pr
JOIN physionet-data.mimiciii_clinical.diagnoses_icd d
    ON pr.subject_id = d.subject_id
WHERE d.icd9_code = '3320'
GROUP BY pr.drug
ORDER BY num_patients_prescribed DESC
LIMIT 10;
''').to_dataframe()


,drug,num_patients_prescribed
0,Sodium Chloride 0.9% Flush,315
1,Acetaminophen,308
2,Potassium Chloride,300
3,Heparin,294
4,Magnesium Sulfate,259
5,Insulin,255
6,Iso-Osmotic Dextrose,254
7,Docusate Sodium,241
8,NS,239
9,D5W,233


### Query 9 - Top Procedures Performed on Parkinson's Patients

In [ ]:
bq_client.query('''
SELECT
    pr.icd9_code,
    dicd.long_title AS procedure_description,
    COUNT(DISTINCT pr.subject_id) AS num_patients
FROM
    `physionet-data.mimiciii_clinical.procedures_icd` pr
JOIN
    `physionet-data.mimiciii_clinical.diagnoses_icd` d
    ON pr.subject_id = d.subject_id
JOIN
    `physionet-data.mimiciii_clinical.d_icd_procedures` dicd
    ON pr.icd9_code = dicd.icd9_code
WHERE
    d.icd9_code = '3320'
GROUP BY
    pr.icd9_code, dicd.long_title
ORDER BY
    num_patients DESC
LIMIT 10;
''').to_dataframe()


,icd9_code,procedure_description,num_patients
0,3893,"Venous catheterization, not elsewhere classified",143
1,9604,Insertion of endotracheal tube,106
2,966,Enteral infusion of concentrated nutritional s...,106
3,9671,Continuous invasive mechanical ventilation for...,90
4,9904,Transfusion of packed cells,84
5,9672,Continuous invasive mechanical ventilation for...,68
6,3891,Arterial catheterization,45
7,8856,Coronary arteriography using two catheters,43
8,3961,Extracorporeal circulation auxiliary to open h...,42
9,4311,Percutaneous [endoscopic] gastrostomy [PEG],35


### Query 10 - In-hospital Mortality Rate of Parkinson's Patients vs. Other Patients

In [ ]:
bq_client.query('''
SELECT
    CASE
        WHEN subject_id IN (
            SELECT subject_id FROM physionet-data.mimiciii_clinical.diagnoses_icd
            WHERE icd9_code = '3320'
        ) THEN 'Parkinsons Patients'
        ELSE 'Other Patients'
    END AS patient_group,
    COUNT(DISTINCT subject_id) AS num_patients,  -- Count distinct subject_id to get unique patients
    SUM(CASE WHEN hospital_expire_flag = 1 THEN 1 ELSE 0 END) AS num_deaths,
    SUM(CASE WHEN hospital_expire_flag = 1 THEN 1 ELSE 0 END) / COUNT(DISTINCT subject_id) AS mortality_rate
FROM physionet-data.mimiciii_clinical.admissions
GROUP BY patient_group;
''').to_dataframe()


,patient_group,num_patients,num_deaths,mortality_rate
0,Other Patients,46123,5770,0.125100
1,Parkinsons Patients,397,84,0.211587
